In [1]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

DEVICE = "cuda"

In [19]:
# Load images
# image1 = load_image("https://media.istockphoto.com/id/485371557/photo/twilight-at-spirit-island.jpg?s=612x612&w=0&k=20&c=FSGliJ4EKFP70Yjpzso0HfRR4WwflC6GKfl4F3Hj7fk=")
# image2 = load_image("https://huggingface.co/spaces/merve/chameleon-7b/resolve/main/bee.jpg")
image1 = Image.open("../../../.vscode/fuji-mountain-in-autumn.jpg")

# Initialize processor and model
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")

# Initialize model directly on CUDA with Flash Attention 2.0
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-Instruct",
    torch_dtype=torch.bfloat16,
    _attn_implementation="flash_attention_2",
    device_map="cuda",
)

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            # {"type": "image"},
            {"type": "text", "text": "Can you describe the image?"}
        ]
    },
]

In [ ]:
# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = inputs.to("cuda")

# Generate outputs
generated_ids = model.generate(
    **inputs,
    max_new_tokens=500,
    do_sample=False,  # Use greedy decoding (highest logit)
)
generated_texts = processor.batch_decode(
    generated_ids,  # Slice to include only the first token
    skip_special_tokens=True,
)

print(">>", generated_texts[0])

>> User:<image>Can you describe the image?
Assistant: The image features a scenic view of a mountain, likely Mount Fuji in Japan, with a bridge in the foreground and a body of water in the foreground. The mountain is partially covered with snow, indicating it is likely in the winter season. The sky is clear with a few clouds, and the water in the foreground is calm and still, reflecting the colors of the sky and the mountain. The bridge is a long, narrow structure that spans the body of water, and it appears to be a traditional wooden bridge with a simple design. The bridge is surrounded by trees, which are mostly green but have some hints of autumn foliage, suggesting the season is changing. The trees are reflected in the water, adding to the scenic beauty of the image.


In [ ]:
"""
>> User:<image>Can you describe the image?
Assistant: The image features a scenic view of a mountain, likely Mount Fuji in Japan, with a bridge in the foreground and a body of water in the foreground. The mountain is partially covered with snow, indicating it is likely in the winter season. The sky is clear with a few clouds, and the water in the foreground is calm and still, reflecting the colors of the sky and the mountain. The bridge is a long, narrow structure that spans the body of water, and it appears to be a traditional wooden bridge with a simple design. The bridge is surrounded by trees, which are mostly green but have some hints of autumn foliage, suggesting the season is changing. The trees are reflected in the water, adding to the scenic beauty of the image.
"""